In [ ]:
!pip install --upgrade albumentations

In [ ]:
%matplotlib inline

In [ ]:
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A
import pandas as pd
import numpy as np
import json
import os
#https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, class_name, color=BOX_COLOR, thickness=2):
    """Visualizes a single bounding box on the image"""
    x_min, y_min, w, h = bbox
    x_min, x_max, y_min, y_max = int(x_min), int(x_min + w), int(y_min), int(y_min + h)
   
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    
    ((text_width, text_height), _) = cv2.getTextSize(class_name, cv2.FONT_HERSHEY_SIMPLEX, 0.35, 1)    
    cv2.rectangle(img, (x_min, y_min - int(1.3 * text_height)), (x_min + text_width, y_min), BOX_COLOR, -1)
    cv2.putText(
        img,
        text=class_name,
        org=(x_min, y_min - int(0.3 * text_height)),
        fontFace=cv2.FONT_HERSHEY_SIMPLEX,
        fontScale=0.35, 
        color=TEXT_COLOR, 
        lineType=cv2.LINE_AA,
    )
    return img


def visualize(image, bboxes, category_ids, category_id_to_name):
    img = image.copy()
    for bbox, category_id in zip(bboxes, category_ids):
        class_name = category_id_to_name[category_id]
        img = visualize_bbox(img, bbox, class_name)
    plt.figure(figsize=(20, 20))
    plt.axis('off')
    plt.imshow(img)

In [ ]:
CSV_PATH = "/content/drive/MyDrive/Tesis/Datasets/Products/annotations.csv"
IMAGES_PATH = "/content/drive/MyDrive/Tesis/Datasets/Products/Images/"

In [ ]:

def pre_process_data(path,images_path =""):
  colnames=['filename','xmin', 'ymin', 'xmax', 'ymax',\
            'class','width','height']
  
  annotate_data=pd.read_csv(path,header=None)
  annotate_data.columns = colnames
  annotate_data['filename'] = images_path+annotate_data['filename'].astype(str)
  annotate_data = annotate_data[['filename', 'class','width', 'height', 'xmin','ymin','xmax','ymax']]
  return annotate_data

images_df = pre_process_data(CSV_PATH,IMAGES_PATH)

#CSV TO COCO

In [ ]:
print("Number of different images: {0}".format(len(images_df['filename'].unique())))
images_df.head()

Number of different images: 143


,filename,class,width,height,xmin,ymin,xmax,ymax
0,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,263,173,598,729
1,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,610,87,789,747
2,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,798,155,959,753
3,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,965,170,1144,747
4,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,1141,164,1332,759


In [ ]:
images_df.tail()

,filename,class,width,height,xmin,ymin,xmax,ymax
7068,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,605,1834,928,2314
7069,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,495,2936,808,3110
7070,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,482,3122,798,3279
7071,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,495,3296,808,3453
7072,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,2265,3421,2723,4014


In [ ]:
def image(row):
    image = {}
    image["height"] = row.height
    image["width"] = row.width
    image["id"] = row.fileid
    image["file_name"] = row.filename
    return image

def category(row):
    category = {}
    category["supercategory"] = 'None'
    category["id"] = row.categoryid
    category["name"] = row[2]
    return category

def annotation(row):
    annotation = {}
    area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
    annotation["segmentation"] = []
    annotation["iscrowd"] = 0
    annotation["area"] = area
    annotation["image_id"] = row.fileid

    annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

    annotation["category_id"] = row.categoryid
    annotation["id"] = row.annid
    return annotation


def csv_to_coco(data,save_json_path):

    images = []
    categories = []
    annotations = []

    # _category = {}
    # _category["supercategory"] = 'none'
    # _category["id"] = 0
    # _category["name"] = 'None'
    # categories.append(_category)

    data['fileid'] = data['filename'].astype('category').cat.codes
    data['categoryid']= pd.Categorical(data['class'],ordered= True).codes
    data['categoryid'] = data['categoryid']+1
    data['annid'] = data.index

    for row in data.itertuples():
        annotations.append(annotation(row))

    imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
    for row in imagedf.itertuples():
        images.append(image(row))

    catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
    for row in catdf.itertuples():
        categories.append(category(row))

    data_coco = {}
    data_coco["images"] = images
    data_coco["categories"] = categories
    data_coco["annotations"] = annotations


    json.dump(data_coco, open(save_json_path, "w"), indent=4)

IMAGES_JSON_PATH = '/content/images_coco.json'

csv_to_coco(images_df.copy(),IMAGES_JSON_PATH)


In [ ]:
def read_json(filename):
  with open(filename) as json_file:
        data = json.load(json_file)
  annotations = {data['images'][i]['id']:[data['images'][i]['file_name'],[],[]] for i in range(len(data['images']))}
  #print(annotations)
  for d in data['annotations']:
    annotations[d['image_id']][1].append(list(d['bbox']))
    annotations[d['image_id']][2].append(d['category_id'])
  return annotations
    



anns = read_json(IMAGES_JSON_PATH)

ids = {'object':0,'fluo_cardent':1,'zucaritas':2,'chocokrispis':3,'fruit_loops':4,'colombiana':5,'manzana':6,'pepsi':7,'corn_flakes':8,'papas_limon':9,'colgate':10,'papas_pollo':11,'papas_natural':12}
for k in ids.keys():
    ids[k] += 1

category_id_to_name = {ids[k]:k for k in ids.keys()}
#images_names = [f for f in os.listdir(IMAGES_PATH) if f.endswith('jpg')]
#images_names

In [ ]:
N = 2
anns_keys = list(anns.keys())
for i in range(N):
  k = anns_keys[i]
  image,bboxes,category_ids = cv2.imread(anns[k][0]),anns[k][1],anns[k][2]
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  visualize(image,bboxes,category_ids,category_id_to_name)


Output hidden; open in https://colab.research.google.com to view.

#Augmentation


In [ ]:
#annotate_data = annotate_data[['filename', 'class','width', 'height', 'xmin','ymin','xmax','ymax']]
  
def save_transformed_image(transformed_image,original_image,images_path,suffix,save_images_path):
  global images_df
  file_name = original_image[0].replace(images_path,'')
  new_file_name = file_name.replace('.jpg',suffix+'.jpg')
  image,bboxes,category_ids = transformed['image'],transformed['bboxes'],transformed['category_ids']
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  for i in range(len(bboxes)):
    xmin, ymin, w, h = bboxes[i]
    xmin, xmax, ymin, ymax = int(xmin), int(xmin + w), int(ymin), int(ymin + h)
    w,h = int(image.shape[1]),int(image.shape[0])
    #images_df.append({'filename':images_path+new_file_name,'class':category_ids[i],'width':w,'height':h,'xmin':xmin,
    #                  'ymin':ymin,'xmax':xmax,'ymax':ymax},ignore_index=True)
    images_df.loc[images_df.shape[0]] = [images_path+new_file_name,category_ids[i],w,h,xmin,ymin,xmax,ymax]
  cv2.imwrite(save_images_path+new_file_name,image)
  




#Crops

##2048x2048

In [ ]:
#random.seed(7)
transform = A.Compose([
    A.RandomCrop(width=2048, height=2048),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
)

In [ ]:
!mkdir random_crops2048x2048

In [ ]:

anns_keys = list(anns.keys())
N = len(anns_keys)
#N = 4
suffix = "random_crop_2048x2048"
save_images_path = "/content/random_crops2048x2048/"

for i in range(N):
  k = anns_keys[i]
  image,bboxes,category_ids = cv2.imread(anns[k][0]),anns[k][1],anns[k][2]
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  transformed = transform(image=image,bboxes=bboxes, category_ids=category_ids)
  save_transformed_image(transformed,anns[k],IMAGES_PATH,suffix,save_images_path)
  if(i < 4):
    visualize(
      transformed['image'],
      transformed['bboxes'],
      transformed['category_ids'],
      category_id_to_name,
    )
  


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("Number of different images: {0}".format(len(images_df['filename'].unique())))
images_df.tail()

Number of different images: 286


,filename,class,width,height,xmin,ymin,xmax,ymax
11083,/content/drive/MyDrive/Tesis/Datasets/Products...,8,2048,2048,1601,489,1892,1405
11084,/content/drive/MyDrive/Tesis/Datasets/Products...,1,2048,2048,1904,557,2048,1463
11085,/content/drive/MyDrive/Tesis/Datasets/Products...,1,2048,2048,0,1150,82,1762
11086,/content/drive/MyDrive/Tesis/Datasets/Products...,1,2048,2048,109,1279,405,1923
11087,/content/drive/MyDrive/Tesis/Datasets/Products...,1,2048,2048,812,1373,1204,2048


In [ ]:
!zip -r '/content/random_crops2048x2048.zip' '/content/random_crops2048x2048'

  adding: content/random_crops2048x2048/ (stored 0%)
  adding: content/random_crops2048x2048/IMG_1402random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1260random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1297random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1283random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1319random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1415random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1325random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1427random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1348random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1339random_crop_2048x2048.jpg (deflated 0%)
  adding: content/random_crops2048x2048/IMG_1353random_crop_2048x20

In [ ]:
transform = A.Compose([
    A.CenterCrop(width=2048, height=2048),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
)

In [ ]:
!mkdir center_crops2048x2048

In [ ]:

anns_keys = list(anns.keys())
N = len(anns_keys)
#N = 4
suffix = "center_crop_2048x2048"
save_images_path = "/content/center_crops2048x2048/"

for i in range(N):
  k = anns_keys[i]
  image,bboxes,category_ids = cv2.imread(anns[k][0]),anns[k][1],anns[k][2]
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  transformed = transform(image=image,bboxes=bboxes, category_ids=category_ids)
  save_transformed_image(transformed,anns[k],IMAGES_PATH,suffix,save_images_path)
  if(i < 4):
    visualize(
      transformed['image'],
      transformed['bboxes'],
      transformed['category_ids'],
      category_id_to_name,
    )
  


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("Number of different images: {0}".format(len(images_df['filename'].unique())))
images_df.tail()

Number of different images: 429


,filename,class,width,height,xmin,ymin,xmax,ymax
15283,/content/drive/MyDrive/Tesis/Datasets/Products...,6,2048,2048,1040,1246,1263,2048
15284,/content/drive/MyDrive/Tesis/Datasets/Products...,8,2048,2048,1273,1315,1564,2048
15285,/content/drive/MyDrive/Tesis/Datasets/Products...,1,2048,2048,1576,1383,1821,2048
15286,/content/drive/MyDrive/Tesis/Datasets/Products...,1,2048,2048,251,115,498,507
15287,/content/drive/MyDrive/Tesis/Datasets/Products...,1,2048,2048,1831,1400,2048,2048


In [ ]:
!zip -r '/content/center_crops2048x2048.zip' '/content/center_crops2048x2048'

  adding: content/center_crops2048x2048/ (stored 0%)
  adding: content/center_crops2048x2048/IMG_1266center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1298center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1435center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1294center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1428center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1313center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1339center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1438center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1405center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1307center_crop_2048x2048.jpg (deflated 0%)
  adding: content/center_crops2048x2048/IMG_1417center_crop_2048x20

##1024x1024

In [ ]:
#random.seed(7)
transform = A.Compose([
    A.RandomCrop(width=1024, height=1024,p=1.0),
    A.RandomBrightnessContrast(p=0.2),
], bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
)

In [ ]:
!mkdir crops1024x1024

In [ ]:

anns_keys = list(anns.keys())
N = len(anns_keys)
#N = 4
suffix = "crop_1024x1024"
save_images_path = "/content/crops1024x1024/"
for i in range(N):
  k = anns_keys[i]
  image,bboxes,category_ids = cv2.imread(anns[k][0]),anns[k][1],anns[k][2]
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  transformed = transform(image=image,bboxes=bboxes, category_ids=category_ids)
  save_transformed_image(transformed,anns[k],IMAGES_PATH,suffix,save_images_path)
  if(i < 4):
    visualize(
      transformed['image'],
      transformed['bboxes'],
      transformed['category_ids'],
      category_id_to_name,
    )
  
  #visualize(image,bboxes,category_ids,category_id_to_name)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("Number of different images: {0}".format(len(images_df['filename'].unique())))
images_df.tail()

Number of different images: 569


,filename,class,width,height,xmin,ymin,xmax,ymax
16541,/content/drive/MyDrive/Tesis/Datasets/Products...,7,1024,1024,0,0,5,83
16542,/content/drive/MyDrive/Tesis/Datasets/Products...,6,1024,1024,13,0,236,144
16543,/content/drive/MyDrive/Tesis/Datasets/Products...,8,1024,1024,246,0,537,248
16544,/content/drive/MyDrive/Tesis/Datasets/Products...,1,1024,1024,549,0,794,306
16545,/content/drive/MyDrive/Tesis/Datasets/Products...,1,1024,1024,804,0,1024,352


In [ ]:
!zip -r '/content/crops1024x1024.zip' '/content/crops1024x1024'

  adding: content/crops1024x1024/ (stored 0%)
  adding: content/crops1024x1024/IMG_1260crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1419crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1336crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1350crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1330crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1293crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1405crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1410crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1308crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1416crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1304crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1316crop_1024x1024.jpg (deflated 0%)
  adding: content/crops1024x1024/IMG_1319crop_1024x1024.jpg (defla

#Gaussian filters

In [ ]:
transform = A.Compose(
    [A.GaussianBlur(p=1.0,blur_limit=(15,35),always_apply=True)],
    bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
)

In [ ]:
!mkdir gaussian_blur

In [ ]:

anns_keys = list(anns.keys())
N = len(anns_keys)
#N = 4
suffix = "gaussian_blur"
save_images_path = "/content/gaussian_blur/"
for i in range(N):
  k = anns_keys[i]
  image,bboxes,category_ids = cv2.imread(anns[k][0]),anns[k][1],anns[k][2]
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  transformed = transform(image=image,bboxes=bboxes, category_ids=category_ids)
  save_transformed_image(transformed,anns[k],IMAGES_PATH,suffix,save_images_path)
  if(i < 4):
    visualize(
      transformed['image'],
      transformed['bboxes'],
      transformed['category_ids'],
      category_id_to_name,
    )
  
  #visualize(image,bboxes,category_ids,category_id_to_name)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("Number of different images: {0}".format(len(images_df['filename'].unique())))
images_df.tail()

Number of different images: 712


,filename,class,width,height,xmin,ymin,xmax,ymax
23614,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,269,3097,565,3741
23615,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,2356,20,2946,662
23616,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,739,1107,986,1499
23617,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,2319,2392,2608,3327
23618,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,972,3191,1364,3972


In [ ]:
!zip -r '/content/gaussian_blur.zip' '/content/gaussian_blur'

  adding: content/gaussian_blur/ (stored 0%)
  adding: content/gaussian_blur/IMG_1336gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1334gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1343gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1435gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1315gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1423gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1411gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1352gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1351gaussian_blur.jpg (deflated 1%)
  adding: content/gaussian_blur/IMG_1279gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1347gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1326gaussian_blur.jpg (deflated 0%)
  adding: content/gaussian_blur/IMG_1325gaussian_blur.jpg (deflated 0%)
  adding: content/g

#Random brightness

In [ ]:
transform = A.Compose(
    [A.RandomBrightnessContrast(brightness_limit=[-0.4,0.7], p=1.0)],
    bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
)

In [ ]:
!mkdir brightness

In [ ]:

anns_keys = list(anns.keys())
N = len(anns_keys)
#N = 4
suffix = "brightness"
save_images_path = "/content/brightness/"
for i in range(N):
  k = anns_keys[i]
  image,bboxes,category_ids = cv2.imread(anns[k][0]),anns[k][1],anns[k][2]
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#RGB2GRAY
  transformed = transform(image=image,bboxes=bboxes, category_ids=category_ids)
  save_transformed_image(transformed,anns[k],IMAGES_PATH,suffix,save_images_path)
  if(i < 4):
    visualize(
      transformed['image'],
      transformed['bboxes'],
      transformed['category_ids'],
      category_id_to_name,
    )
  
  #visualize(image,bboxes,category_ids,category_id_to_name)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("Number of different images: {0}".format(len(images_df['filename'].unique())))
images_df.tail()

Number of different images: 855


,filename,class,width,height,xmin,ymin,xmax,ymax
30687,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,269,3097,565,3741
30688,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,2356,20,2946,662
30689,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,739,1107,986,1499
30690,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,2319,2392,2608,3327
30691,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,972,3191,1364,3972


In [ ]:
!zip -r '/content/brightness.zip' '/content/brightness'

  adding: content/brightness/ (stored 0%)
  adding: content/brightness/IMG_1333brightness.jpg (deflated 1%)
  adding: content/brightness/IMG_1399brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1340brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1350brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1428brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1286brightness.jpg (deflated 1%)
  adding: content/brightness/IMG_1329brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1331brightness.jpg (deflated 1%)
  adding: content/brightness/IMG_1407brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1405brightness.jpg (deflated 1%)
  adding: content/brightness/IMG_1413brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1285brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1411brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1322brightness.jpg (deflated 0%)
  adding: content/brightness/IMG_1

#Colors

##Gray

In [ ]:
transform = A.Compose(
    [A.ToGray(p=1.0)],
    bbox_params=A.BboxParams(format='coco', label_fields=['category_ids']),
)

In [ ]:
!mkdir gray

In [ ]:
anns_keys = list(anns.keys())
N = len(anns_keys)
#N = 4
suffix = "gray"
save_images_path = "/content/gray/"
for i in range(N):
  k = anns_keys[i]
  image,bboxes,category_ids = cv2.imread(anns[k][0]),anns[k][1],anns[k][2]
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)#RGB2GRAY
  transformed = transform(image=image,bboxes=bboxes, category_ids=category_ids)
  save_transformed_image(transformed,anns[k],IMAGES_PATH,suffix,save_images_path)
  if(i < 4):
    visualize(
      transformed['image'],
      transformed['bboxes'],
      transformed['category_ids'],
      category_id_to_name,
    )
  
  #visualize(image,bboxes,category_ids,category_id_to_name)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
print("Number of different images: {0}".format(len(images_df['filename'].unique())))
images_df.tail()

Number of different images: 998


,filename,class,width,height,xmin,ymin,xmax,ymax
37760,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,269,3097,565,3741
37761,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,2356,20,2946,662
37762,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,739,1107,986,1499
37763,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,2319,2392,2608,3327
37764,/content/drive/MyDrive/Tesis/Datasets/Products...,1,3024,4032,972,3191,1364,3972


In [ ]:
!zip -r '/content/gray.zip' '/content/gray'

  adding: content/gray/ (stored 0%)
  adding: content/gray/IMG_1324gray.jpg (deflated 0%)
  adding: content/gray/IMG_1435gray.jpg (deflated 0%)
  adding: content/gray/IMG_1431gray.jpg (deflated 0%)
  adding: content/gray/IMG_1303gray.jpg (deflated 0%)
  adding: content/gray/IMG_1415gray.jpg (deflated 0%)
  adding: content/gray/IMG_1419gray.jpg (deflated 0%)
  adding: content/gray/IMG_1342gray.jpg (deflated 0%)
  adding: content/gray/IMG_1294gray.jpg (deflated 0%)
  adding: content/gray/IMG_1353gray.jpg (deflated 0%)
  adding: content/gray/IMG_1288gray.jpg (deflated 0%)
  adding: content/gray/IMG_1269gray.jpg (deflated 0%)
  adding: content/gray/IMG_1336gray.jpg (deflated 0%)
  adding: content/gray/IMG_1309gray.jpg (deflated 0%)
  adding: content/gray/IMG_1264gray.jpg (deflated 0%)
  adding: content/gray/IMG_1314gray.jpg (deflated 0%)
  adding: content/gray/IMG_1311gray.jpg (deflated 0%)
  adding: content/gray/IMG_1298gray.jpg (deflated 0%)
  adding: content/gray/IMG_1416gray.jpg (defla

#Save csv with augmented data

In [ ]:
images_df["filename"] = images_df["filename"].apply(lambda x: x.replace(IMAGES_PATH,''))
images_df.head()

,filename,class,width,height,xmin,ymin,xmax,ymax
0,IMG_1259.jpg,1,3024,4032,263,173,598,729
1,IMG_1259.jpg,1,3024,4032,610,87,789,747
2,IMG_1259.jpg,1,3024,4032,798,155,959,753
3,IMG_1259.jpg,1,3024,4032,965,170,1144,747
4,IMG_1259.jpg,1,3024,4032,1141,164,1332,759


In [ ]:
print("Number of different images: {0}".format(len(images_df['filename'].unique())))

Number of different images: 998


In [ ]:
FINAL_CSV = "/content/annotations_data_augmented.csv"

# #Save the dataframes to memory
images_df.to_csv(FINAL_CSV, index=False)